In [ ]:
# importing the zipfile module
from zipfile import ZipFile

# loading the temp.zip and creating a zip object
with ZipFile("/content/AD.zip", 'r') as zObject:

    # Extracting all the members of the zip
    # into a specific location.
    zObject.extractall(
        path="/content")

In [ ]:
# importing the zipfile module
from zipfile import ZipFile

# loading the temp.zip and creating a zip object
with ZipFile("/content/TD.zip", 'r') as zObject:

    # Extracting all the members of the zip
    # into a specific location.
    zObject.extractall(
        path="/content")

repeat for combination (C3, Cz) and (C3,C4)

In [ ]:
!pip install pywavelets
import pandas as pd
ns =11
ori_data = dict()
mod_data = dict()
ori_data1 = dict()
mod_data1 = dict()
# function to count subject
def subjectCounter(i):
    return 'subject0{}'.format(i)
for i in range(1, ns):
    data_path = '/content/AD/AD/AV{:02d}.xlsx'.format(i)
    subj = subjectCounter(i)
    ori_data[subj] =  pd.read_excel(data_path, usecols=['C4','Cz'])
    mod_data[subj] = {}
    mod_data[subj]['raw_EEG'] = ori_data[subj]
for i in range(1, ns):
    data_path = '/content/TD/TD{:02d}.xlsx'.format(i)
    subj = subjectCounter(i)
    ori_data1[subj] =  pd.read_excel(data_path, usecols=['C4','Cz'])
    mod_data1[subj] = {}
    mod_data1[subj]['raw_EEG'] = ori_data1[subj]

In [ ]:
from scipy import signal
for i in range(1, ns):
    subj = subjectCounter(i)
    mod_data[subj] = {}
for i in range(1, ns):
    subj = subjectCounter(i)
    mod_data1[subj] = {}
for i in range(1, ns):
    subj = subjectCounter(i)
    mod_data[subj]['raw_EEG'] = ori_data[subj]
for i in range(1, ns):
    subj = subjectCounter(i)
    mod_data1[subj]['raw_EEG'] = ori_data1[subj]
from scipy.signal import butter, lfilter
from scipy.signal import freqz
from scipy.signal import butter, filtfilt
def butter_bandpass_filter(signal, lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut/nyq
    high = highcut/nyq
    b,a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, signal)
    return y
for i in range(1, ns):
    subj = subjectCounter(i)
    mod_data[subj]['raw_EEG'] = mod_data[subj]['raw_EEG'].T
for i in range(1, ns):
    subj = subjectCounter(i)
    mod_data1[subj]['raw_EEG'] = mod_data1[subj]['raw_EEG'].T
import pywt
from scipy import signal
import numpy as np
def dwt_bandpass_one_subject(data, subj, lowcut, highcut, fs, interval=None):
    data[subj]['EEG_filtered'] = {}
    temp_raw_EEG = data[subj]['raw_EEG']
    temp_band = 'temp_band'  # Replace 'your_desired_key' with the actual key in your dictionary
    if interval is not None:
        startband = np.arange(lowcut, highcut, step=interval)
        for start in startband:
            temp_band = "{:02d}_{:02d}".format(start, start+interval)
            data[subj]['EEG_filtered']['temp_band'] = {}
            data[subj]['EEG_filtered']['temp_band'] = dwt_bandpass_filter(temp_raw_EEG, start, start+interval, fs)
    else:
        temp_band = "{:02d}_{:02d}".format(lowcut, highcut)
        data[subj]['EEG_filtered']['temp_band'] = {}
        data[subj]['EEG_filtered']['temp_band'] = dwt_bandpass_filter(temp_raw_EEG, lowcut, highcut, fs)
def dwt_bandpass_filter(data, lowcut, highcut, fs):
    coeffs = pywt.wavedec(data, 'db4', level=3)
    bands = [(0.5, 32)]
    subband_filters = []
    for low, high in bands:
        b, a = signal.butter(5, [low/(fs/2), high/(fs/2)], btype='bandpass')
    for i in range(1, 4):
        for j in range(0, 3):
            coeffs[i][j] = signal.filtfilt(b, a, coeffs[i][j])
    filtered_data = pywt.waverec(coeffs, 'db4')
    return filtered_data
lowcut=8
highcut=32
fs = 500
temp_band = 'temp_band'
for i in range(1, 11):
    subj = subjectCounter(i)
    dwt_bandpass_one_subject(mod_data, subj, lowcut, highcut, fs, interval = None)
lowcut=8
highcut=32
fs = 500
for i in range(1, 11):
    subj = subjectCounter(i)
    dwt_bandpass_one_subject(mod_data1, subj, lowcut, highcut, fs, interval = None)
def butter_bandpass_one_subject(data, subj, lowcut, highcut, fs, interval=None):
    data[subj]['filtered'] = {}
    temp_raw_EEG = data[subj]['EEG_filtered']['temp_band']
    if interval is not None:
        startband = np.arange(lowcut, highcut, step = interval)
        for start in startband:
            band = "{:02d}_{:02d}".format(start, start+interval)
            data[subj]['filtered'][band] = {}
            data[subj]['filtered'][band]['EEG_all'] = butter_bandpass_filter(temp_raw_EEG, start, start+interval, fs, order=5)
    else:
        band = "{:02d}_{:02d}".format(lowcut, highcut)
        data[subj]['EEG_filtered'][band]['EEG_all'] = butter_bandpass_filter(temp_raw_EEG, lowcut, highcut, fs)
lowcut=8
highcut=32
fs = 500
for i in range(1, 11):
    subj = subjectCounter(i)
    butter_bandpass_one_subject(mod_data, subj, lowcut, highcut, fs, interval = 4)
lowcut=8
highcut=32
fs = 500
for i in range(1, 11):
    subj = subjectCounter(i)
    butter_bandpass_one_subject(mod_data1, subj, lowcut, highcut, fs, interval = 4)

In [ ]:
ASD = 'ASD'
for i in range(1, 11):
    subj = subjectCounter(i)
    print('Processing ', subj)
    for band in mod_data[subj]['filtered'].keys():
        temp_EEG_all = mod_data[subj]['filtered'][band]['EEG_all']
        temp_ASD = []
        for j in range(0,61):
            subset_size = 1000
            temp_ASD.append(temp_EEG_all[:,j * subset_size:(j+1) * subset_size])
        mod_data[subj]['filtered'][band]['ASD'] = np.array(temp_ASD)
TD = 'TD'
for i in range(1, 11):
    subj = subjectCounter(i)
    print('Processing ', subj)
    for band in mod_data1[subj]['filtered'].keys():
        temp_EEG_all = mod_data1[subj]['filtered'][band]['EEG_all']
        temp_TD = []
        for j in range(0,61):
            subset_size = 1000
            temp_TD.append(temp_EEG_all[:,j * subset_size:(j+1) * subset_size])
        mod_data1[subj]['filtered'][band]['TD'] = np.array(temp_TD)
def split_EEG_one_class(EEG_one_class, percent_train=0.8):
    n = EEG_one_class.shape[0]
    n_tr = round(n*percent_train)
    n_te = n - n_tr
    EEG_train = EEG_one_class[:n_tr]
    EEG_test = EEG_one_class[n_tr:n_tr+n_te]
    return EEG_train, EEG_test
for i in range(1, 11):
    subj = subjectCounter(i)
    for band in mod_data[subj]['filtered'].keys():
        temp_EEG_ASD = mod_data[subj]['filtered'][band]['EEG_ASD']
        temp_filt = mod_data[subj]['filtered'][band]
        temp_filt['EEG_ASD_train'], temp_filt['EEG_ASD_test'] = split_EEG_one_class(temp_EEG_ASD, 0.8)
        print(temp_filt['EEG_ASD_test'])
for i in range(1, 11):
    subj = subjectCounter(i)
    for band in mod_data1[subj]['filtered'].keys():
        temp_EEG_TD = mod_data1[subj]['filtered'][band]['EEG_TD']
        temp_filt = mod_data1[subj]['filtered'][band]
        temp_filt['EEG_TD_train'], temp_filt['EEG_TD_test'] = split_EEG_one_class(temp_EEG_TD, 0.8)
for i in range(1, ns):
    subj = subjectCounter(i)
    mod_data[subj]['CSP'] = {}
for i in range(1, ns):
    subj = subjectCounter(i)
    mod_data1[subj]['CSP'] = {}

In [ ]:
import numpy as np
from sklearn.covariance import GraphicalLasso, GraphicalLassoCV
def compute_cov(EEG_data):
    cov = []
    alpha = 0.1
    for i in range(EEG_data.shape[0]):
        cov.append(EEG_data[i]@EEG_data[i].T/np.trace(EEG_data[i]@EEG_data[i].T))
    cov = np.mean(np.array(cov), 0)
    return cov
for i in range(1, 11):
    subj = subjectCounter(i)
    for band in mod_data[subj]['filtered'].keys():
        temp_band = mod_data[subj]['CSP'][band] = {}
        temp_band1 = mod_data1[subj]['CSP'][band] = {}
        temp_band['cov_ASD'] = compute_cov(mod_data[subj]['filtered'][band]['EEG_ASD_train'])
        temp_band1['cov_TD'] = compute_cov(mod_data1[subj]['filtered'][band]['EEG_TD_train'])
        temp_band['cov_comp'] = temp_band['cov_ASD'] + temp_band1['cov_TD']
        print(temp_band['cov_comp'])

In [ ]:
for i in range(1, 11):
    subj = subjectCounter(i)
    for band in mod_data[subj]['filtered'].keys():
        mod_data[subj]['CSP'][band]['whitening'] = {}
from scipy.linalg import sqrtm
from scipy.linalg import inv
def decompose_cov(avg_cov):
    λ, V = np.linalg.eig(avg_cov)
    λ_dsc = np.sort(λ)[::-1]
    idx_dsc = np.argsort(λ)[::-1]
    V_dsc = V[:, idx_dsc]
    λ_dsc = np.diag(λ_dsc)
    return λ_dsc, V_dsc
def white_matrix(λ_dsc, V_dsc):
    λ_dsc_sqr = sqrtm(inv(λ_dsc))
    P = (λ_dsc_sqr)@(V_dsc.T)
    return P
for i in range(1, 11):
    subj = subjectCounter(i)
    for band in mod_data[subj]['filtered'].keys():
        temp_whitening = mod_data[subj]['CSP'][band]['whitening']
        temp_cov = mod_data[subj]['CSP'][band]['cov_comp']
        temp_whitening['eigval'], temp_whitening['eigvec'] = decompose_cov(temp_cov)
        temp_whitening['P'] = white_matrix(temp_whitening['eigval'], temp_whitening['eigvec'])
        print(temp_whitening['eigvec'])

In [ ]:
for i in range(1, 11):
    subj = subjectCounter(i)
    for band in mod_data[subj]['filtered'].keys():
        mod_data[subj]['CSP'][band]['S_ASD'] = {}
for i in range(1, 11):
    subj = subjectCounter(i)
    for band in mod_data1[subj]['filtered'].keys():
        mod_data1[subj]['CSP'][band]['S_TD'] = {}
def compute_S(avg_Cov, white):
    S = white@avg_Cov@white.T
    return S
def decompose_S(S_one_class, order='descending'):
    λ, B = np.linalg.eig(S_one_class)
    if order == 'ascending':
        idx = λ.argsort()
    elif order == 'descending':
        idx = λ.argsort()[::-1]
    else:
        print('Wrong order input')
    λ = λ[idx]
    B = B[:, idx]
    return B, λ
for i in range(1, 11):
    subj = subjectCounter(i)
    for band in mod_data[subj]['filtered'].keys():
        temp_P = mod_data[subj]['CSP'][band]['whitening']['P']
        Cl = mod_data[subj]['CSP'][band]['cov_ASD']
        temp_Sl = mod_data[subj]['CSP'][band]['S_ASD']
        Sl = compute_S(Cl, temp_P)
        temp_Sl['eigvec'], temp_Sl['eigval'] = decompose_S(Sl, 'descending')
for i in range(1, 11):
    subj = subjectCounter(i)
    for band in mod_data1[subj]['filtered'].keys():
        temp_P = mod_data[subj]['CSP'][band]['whitening']['P']
        Cr = mod_data1[subj]['CSP'][band]['cov_TD']
        temp_Sr = mod_data1[subj]['CSP'][band]['S_TD']
        Sr = compute_S(Cr, temp_P)
        temp_Sr['eigvec'], temp_Sr['eigval'] = decompose_S(Sr, 'ascending')
        print(temp_Sr['eigval'])

In [ ]:
def spatial_filter(B, P):
    return (B.T@P)
for i in range(1, 11):
    subj = subjectCounter(i)
    for band in mod_data[subj]['filtered'].keys():
        temp_eigvec = mod_data[subj]['CSP'][band]['S_ASD']['eigvec']
        temp_P = mod_data[subj]['CSP'][band]['whitening']['P']
        mod_data[subj]['CSP'][band]['W'] = spatial_filter(temp_eigvec, temp_P)
for i in range(1, 11):
    subj = subjectCounter(i)
    for band in mod_data1[subj]['filtered'].keys():
        temp_eigvec = mod_data1[subj]['CSP'][band]['S_TD']['eigvec']
        temp_P = mod_data[subj]['CSP'][band]['whitening']['P']
        mod_data1[subj]['CSP'][band]['W'] = spatial_filter(temp_eigvec, temp_P)

In [ ]:
for i in range(1, 11):
    subj = subjectCounter(i)
    mod_data[subj]['train'] = {}
    mod_data[subj]['test'] = {}
    for band in mod_data[subj]['filtered'].keys():
        mod_data[subj]['train'][band] = {}
        mod_data[subj]['test'][band] = {}
for i in range(1, 11):
    subj = subjectCounter(i)
    mod_data1[subj]['train'] = {}
    mod_data1[subj]['test'] = {}
    for band in mod_data1[subj]['filtered'].keys():
        mod_data1[subj]['train'][band] = {}
        mod_data1[subj]['test'][band] = {}
m = 2
def compute_Z(W, E, m):
    Z = []
    W = np.delete(W, np.s_[m:-m:], 0)
    for i in range(E.shape[0]):
        Z.append(W @ E[i])
    return np.array(Z)

Repeat feature extraction process then include Stockwell transform and peak frequency with peak-to-peak amplitude feature

In [ ]:
!pip install stockwell
import numpy as np
from stockwell import st  # Assuming this is a valid import for the Stockwell transform
from scipy.signal import welch
def feat_vector(Z):
    feat = []
    fs = 500  # Sampling frequency (adjust as necessary)
    fmin = 8  # Minimum frequency (Hz)
    fmax = 32  # Maximum frequency (Hz)

    for i in range(Z.shape[0]):
        # Calculate peak-to-peak amplitude and its sum
        peak_to_peak = np.ptp(Z[i], axis=1)  # Calculate peak-to-peak amplitude along each signal
        peak_to_peak_sum = np.sum(peak_to_peak)  # Sum of peak-to-peak values for normalization

        # Initialize list to store mean features from Stockwell transform
        matrix_entropy = []
        mean_features_list = []
        peak_frequencies = []

        for signal in Z[i]:
            # Calculate frequency resolution
            signal_length = len(signal)
            df = fs / signal_length  # Frequency step in Hz
            fmin_samples = int(fmin / df)  # Convert frequency to sample index
            fmax_samples = int(fmax / df)  # Convert frequency to sample index

            #Stockwell transform and extract mean features
            stock = st.st(signal)  # Compute Stockwell Transform for the signal
            stock = stock[fmin_samples:fmax_samples]  # Filter frequencies of interest
            mean_features = np.mean(np.abs(stock))  # Mean of absolute values in the frequency range
            mean_features_list.append(mean_features)
            freqs, psd = welch(signal, fs, nperseg=min(len(signal), 256))  # Use a safe nperseg
            #psd_norm = psd / np.sum(psd)  # Normalize the power spectral density
            #spectral_ent = -np.sum(psd_norm * np.log2(psd_norm + 1e-12))  # Compute spectral entropy
            #matrix_entropy.append(spectral_ent)
            band_indices = np.logical_and(freqs >= fmin, freqs <= fmax)
            if np.any(band_indices):  # Ensure there is data in the range
                peak_freq = freqs[band_indices][np.argmax(psd[band_indices])]
                peak_frequencies.append(peak_freq)
        # Normalize peak-to-peak and calculate log10 for each signal
        normalized_peak_to_peak = peak_to_peak / peak_to_peak_sum if peak_to_peak_sum > 0 else np.zeros_like(peak_to_peak)
        log10_peak_to_peak = np.log10(normalized_peak_to_peak, where=normalized_peak_to_peak > 0)  # Avoid log10(0)
        # Append features
        feat.append(np.concatenate((log10_peak_to_peak, [np.mean(peak_frequencies)], [np.mean(mean_features_list)])))
    return np.array(feat)

In [ ]:
for i in range(1, 11):
    subj = subjectCounter(i)
    for band in mod_data[subj]['filtered'].keys():
        temp_W = mod_data[subj]['CSP'][band]['W']
        print(len(temp_W))
        temp_EEG_ASD = mod_data[subj]['filtered'][band]['ASD']
        print(len(temp_EEG_ASD))
        mod_data[subj]['train'][band]['Z_ASD'] = compute_Z(temp_W, temp_EEG_ASD, m)
        mod_data[subj]['train'][band]['feat_ASD'] = feat_vector(mod_data[subj]['train'][band]['Z_ASD'])
        ASD_label = np.zeros([len(mod_data[subj]['train'][band]['feat_ASD']), 1])
    for band in mod_data1[subj]['filtered'].keys():
        temp_W1 = mod_data1[subj]['CSP'][band]['W']
        temp_EEG_ = mod_data1[subj]['filtered'][band]['TD']
        mod_data1[subj]['train'][band]['Z_TD'] = compute_Z(temp_W1, temp_EEG_TD, m)
        mod_data1[subj]['train'][band]['feat_TD'] = feat_vector(mod_data1[subj]['train'][band]['Z_TD'])
        TD_label = np.ones([len(mod_data1[subj]['train'][band]['feat_TD']), 1])
        ASD  = np.c_[mod_data[subj]['train'][band]['feat_ASD'], ASD_label]
        TD  = np.c_[mod_data1[subj]['train'][band]['feat_TD'], TD_label]
        mod_data[subj]['train'][band]['feat_train'] = np.vstack([ASD, TD])
        np.random.shuffle(mod_data[subj]['train'][band]['feat_train'])

In [ ]:
for i in range(1, 11):
    subj = subjectCounter(i)
    feat_ASD_all = []
    feat_TD_all = []
    for band in mod_data[subj]['filtered'].keys():
        feat_ASD = mod_data[subj]['train'][band]['feat_ASD']
        feat_ASD_all.append(feat_ASD)
        feat_TD = mod_data1[subj]['train'][band]['feat_TD']
        feat_TD_all.append(feat_TD)
    merge_ASD = np.zeros(feat_ASD_all[0].shape)
    for i in feat_ASD_all:
        merge_ASD = np.concatenate([merge_ASD, i], axis=1)
    merge_TD = np.zeros(feat_TD_all[0].shape)
    for i in feat_TD_all:
        merge_TD = np.concatenate([merge_TD, i], axis=1)
    true_ASD = np.zeros([merge_ASD.shape[0], 1])
    true_TD = np.ones([merge_TD.shape[0], 1])
    ASD = np.hstack([merge_ASD, true_ASD])
    TD = np.hstack([merge_TD, true_TD])
    train_feat = np.vstack([ASD, TD])
    np.random.shuffle(train_feat)
    mod_data[subj]['train']['all_band'] = train_feat
import numpy as np
from scipy.stats import pearsonr
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif  # You can use f_classif for ANOVA F-statistic as an example
def calculate_correlation_coefficient(X, y):
    correlation_coefficients = []
    for feature in range(X.shape[1]):
        correlation, _ = pearsonr(X[:, feature], y)
        correlation_coefficients.append(correlation)
    return np.array(correlation_coefficients)
all_subjects_data = []
columns = ['Subject'] + [f'Feature_{i+1}' for i in range(4)] + ['Label']

for i in range(1, 11):  # Loop through 10 subjects
    subj = subjectCounter(i)  # Get subject index or identifier
    X_train = mod_data[subj]['train']['all_band'][:, :-1]  # Features
    y_train = mod_data[subj]['train']['all_band'][:, -1]   # Labels

    # Initialize dictionary for mutual features if not already done
    mod_data[subj]['train']['mutual'] = {}

    # Calculate correlation coefficients for feature selection
    correlation_coefficients = calculate_correlation_coefficient(X_train, y_train)

    # Ensure SelectKBest is properly used
    select = SelectKBest(score_func=lambda X, y: correlation_coefficients, k=4)
    X_selected = select.fit_transform(X_train, y_train)

    # Save selected features and labels in `mod_data`
    mod_data[subj]['train']['mutual']['X'] = X_selected
    mod_data[subj]['train']['mutual']['y'] = y_train

    # Append data for each row to all_subjects_data
    for j in range(X_selected.shape[0]):
        row_data = [f'Subject_{subj}'] + X_selected[j, :].tolist() + [y_train[j]]
        all_subjects_data.append(row_data)

# Create DataFrame from collected data
df = pd.DataFrame(all_subjects_data, columns=columns)

# Save to Excel
output_filename = 'selected_train_features_all_subjects.xlsx'
df.to_excel(output_filename, index=False)

np.random.seed(42)
for i in range(1, 11):
    subj = subjectCounter(i)
    for band in mod_data[subj]['filtered'].keys():
        temp_W = mod_data[subj]['CSP'][band]['W']
        temp_EEG_ASD = mod_data[subj]['filtered'][band]['EEG_ASD_test']
        mod_data[subj]['test'][band]['Z_ASD'] = compute_Z(temp_W, temp_EEG_ASD, m)
        mod_data[subj]['test'][band]['feat_ASD'] = feat_vector(mod_data[subj]['test'][band]['Z_ASD'])
        ASD_label = np.zeros([len(mod_data[subj]['test'][band]['feat_ASD']), 1])
    for band in mod_data1[subj]['filtered'].keys():
        temp_W1 = mod_data1[subj]['CSP'][band]['W']
        temp_EEG_TD = mod_data1[subj]['filtered'][band]['EEG_TD_test']
        mod_data1[subj]['test'][band]['Z_TD'] = compute_Z(temp_W1, temp_EEG_TD, m)
        #print(np.shape(mod_data1[subj]['test'][band]['Z_TD'])[0])

        mod_data1[subj]['test'][band]['feat_TD'] = feat_vector(mod_data1[subj]['test'][band]['Z_TD'])
        #print(mod_data1[subj]['test'][band]['feat_TD'])
        TD_label = np.ones([len(mod_data1[subj]['test'][band]['feat_TD']), 1])

        ASD  = np.c_[mod_data[subj]['test'][band]['feat_ASD'], ASD_label]
        TD  = np.c_[mod_data1[subj]['test'][band]['feat_TD'], TD_label]

        mod_data[subj]['test'][band]['feat_test'] = np.vstack([ASD, TD])

        np.random.shuffle(mod_data[subj]['test'][band]['feat_test'])
for i in range(1, 11):
    subj = subjectCounter(i)

    feat_ASD_all = []
    feat_TD_all = []

    for band in mod_data[subj]['filtered'].keys():
        # Access ASD each band
        feat_ASD = mod_data[subj]['test'][band]['feat_ASD']

        feat_ASD_all.append(feat_ASD)

        # Access TD each band
        feat_TD = mod_data1[subj]['test'][band]['feat_TD']

        feat_TD_all.append(feat_TD)

    # MERGING (Need to find more efficient method)
    # ASD
    merge_ASD = np.zeros(feat_ASD_all[0].shape)

    for i in feat_ASD_all:
        merge_ASD = np.concatenate([merge_ASD, i], axis=1)

    # Delete initial zeros
    merge_ASD = np.delete(merge_ASD, np.s_[:2*m], axis=1)

    # TD
    merge_TD = np.zeros(feat_TD_all[0].shape)

    for i in feat_TD_all:
        merge_TD = np.concatenate([merge_TD, i], axis=1)

    # Delete initial zeros
    merge_TD = np.delete(merge_TD, np.s_[:2*m], axis=1)

    # TRUE LABEL
    true_ASD = np.zeros([merge_ASD.shape[0], 1])
    true_TD = np.ones([merge_TD.shape[0], 1])

    # FEATURE + TRUE LABEL
    ASD = np.hstack([merge_ASD, true_ASD])
    TD = np.hstack([merge_TD, true_TD])

    # MERGE ASD AND TD
    test_feat = np.vstack([ASD, TD])

    np.random.shuffle(test_feat)

    mod_data[subj]['test']['all_band'] = test_feat
#print(len(test_feat))
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest
import numpy as np
from scipy.stats import pearsonr
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif# You can use f_classif for ANOVA F-statistic as an example
all_subjects_data = []
columns = ['Subject'] + [f'Feature_{i+1}' for i in range(4)] + ['Label']
# Define a function to calculate the Pearson correlation coefficient
def calculate_correlation_coefficient(X, y):
    correlation_coefficients = []
    for feature in range(X.shape[1]):
        correlation, _ = pearsonr(X[:, feature], y)
        correlation_coefficients.append(correlation)
    return np.array(correlation_coefficients)
for i in range(1, 11):
    subj = subjectCounter(i)

    X_test = mod_data[subj]['test']['all_band'][:, :-1]
    y_test = mod_data[subj]['test']['all_band'][:, -1]

    # New dictionary to store result
    mod_data[subj]['test']['mutual'] = {}

    # Calculate the correlation coefficients for all features
    correlation_coefficients = calculate_correlation_coefficient(X_test, y_test)
    lamda = 0.001
    # Use SelectKBest to select the top 4 features based on correlation coefficient
    select = SelectKBest(score_func=lambda X, y: correlation_coefficients, k=4)
    X_selected = select.fit_transform(X_test, y_test)

    mod_data[subj]['test']['mutual']['X'] = X_selected
    mod_data[subj]['test']['mutual']['y'] = y_test
    for j in range(X_selected.shape[0]):
        row_data = [f'Subject_{subj}'] + X_selected[j, :].tolist() + [y_test[j]]
        all_subjects_data.append(row_data)

# Create DataFrame from collected data
df = pd.DataFrame(all_subjects_data, columns=columns)

# Save to Excel
output_filename = 'selected_test_features_all_subjects.xlsx'
df.to_excel(output_filename, index=False)

In [ ]:
# Quantum SVM Classifier with Data from Excel for a Single Subject
import numpy as np
import pandas as pd
import pennylane as qml
from pennylane.templates.embeddings import AmplitudeEmbedding
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load data for a single subject from Excel files
train_df = pd.read_excel('selected_train_features_all_subjects.xlsx')
test_df = pd.read_excel('selected_test_features_all_subjects.xlsx')

# Separate features and labels (assuming the last column is the label)
X_train = train_df.iloc[:, 1:-1].values  # All columns except the first (subject) and last as features
y_train = train_df.iloc[:, -1].values   # Last column as label
X_test = test_df.iloc[:, 1:-1].values
y_test = test_df.iloc[:, -1].values

# Normalize features
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Quantum SVM setup
n_qubits = X_train.shape[1]
dev_kernel = qml.device("default.qubit", wires=n_qubits)

projector = np.zeros((2 ** n_qubits, 2 ** n_qubits))
projector[0, 0] = 1

@qml.qnode(dev_kernel, interface="autograd")
def kernel(x1, x2):
    """Quantum kernel function."""
    AmplitudeEmbedding(x1, wires=range(n_qubits), pad_with=12.0)
    qml.adjoint(AmplitudeEmbedding)(x2, wires=range(n_qubits), pad_with=12.0)
    return qml.expval(qml.Hermitian(projector, wires=range(n_qubits)))

def kernel_matrix(A, B):
    """Compute kernel matrix for two datasets."""
    return np.array([[kernel(a, b) for b in B] for a in A])

# Train Quantum SVM using custom kernel
quantum_svm = SVC(kernel=lambda A, B: kernel_matrix(A, B)).fit(X_train, y_train)

# Evaluate the classifier
y_pred = quantum_svm.predict(X_test)

# Calculate metrics
test_accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
sensitivity_recall = recall_score(y_test, y_pred)
specificity = recall_score(y_test, y_pred, pos_label=0)
f1 = f1_score(y_test, y_pred)

# Print evaluation metrics
print("Test Set Accuracy:", test_accuracy)
print("Precision:", precision)
print("Sensitivity (Recall):", sensitivity_recall)
print("Specificity:", specificity)
print("F1 Score:", f1)